In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '6'

In [3]:
!pip install pip --upgrade
!pip install albumentations
!python --version

#conda install -c conda-forge albumentations

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
Python 3.10.12


In [4]:
# Download Libraries
!pip install rasterio
!pip install Shapely==1.8.1.post1
!pip install scikit-image
!pip install -U efficientnet_pytorch
!pip install pytorch-metric-learning
!pip install -q timm
!pip install tqdm
!pip install ipywidgets
!pip install torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 583.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.5 MB/s eta 0:00:00
  Attempting uninstall: Shapely
    Found existing installation: shapely 2.0.3
    Uninstalling shapely-2.0.3:
      Successfully uninstalled shapely-2.0.3
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.4 MB/s eta 0:00:00
   ━━━━

In [6]:
import torchvision

# prompt: Import resent.py from /content/drive/MyDrive/Colab Notebooks/multimodal_supercon/model
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/multimodal_supercon/model')

from resnet import *
from effnet import *
from unet import *

ImportError: cannot import name 'model_urls' from 'torchvision.models.resnet' (/usr/local/lib/python3.10/dist-packages/torchvision/models/resnet.py)

In [7]:
#from model.resnet import *
#from model.effnet import *
#from model.unet import *
from utils.metrics import *
from utils.losses import *

import os
import torch
import pandas as pd
import warnings
import numpy as np
from skimage import io, transform
from rasterio.features import rasterize
from shapely.ops import cascaded_union, unary_union
from shapely.geometry import Polygon
from shapely.errors import ShapelyDeprecationWarning
from torch.utils.data import Dataset
import pickle
import albumentations as A

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms, utils, datasets, models
from torch.autograd import Variable
import torch.optim as optim
from torch.optim import lr_scheduler
from pytorch_metric_learning import losses
import copy

from collections import OrderedDict, defaultdict
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import timm
from torch.cuda import amp
from tqdm import tqdm
import torch.optim as optim
from time import time

warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

ERROR: Could not find a version that satisfies the requirement model (from versions: none)
ERROR: No matching distribution found for model


ModuleNotFoundError: No module named 'model'

In [ ]:
# Check file or directory is exist
train_df = pd.read_csv('../../dataset/ForestNetDataset /train.csv')
val_df = pd.read_csv('../../dataset/ForestNetDataset /val.csv')
test_df = pd.read_csv('../../dataset/ForestNetDataset /test.csv')

train_df.head()

In [ ]:
data_array = np.load('../../dataset/ForestNetDataset /examples/4.430849118860583_96.1016343478138/auxiliary/gain.npy')
print(data_array.shape)
fig = plt.figure(figsize = (10, 6))
ax = fig.add_subplot(111)
plt.contour(data_array[0], cmap = "viridis",
            )
plt.title("Gain Contours at (4.430849118860583,96.1016343478138)")
cbar = plt.colorbar()
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
# Plot distribution of data
train_df_class = train_df.groupby(['merged_label']).size().to_frame('train').reset_index()
val_df_class = val_df.groupby(['merged_label']).size().to_frame('val').reset_index()
test_df_class = test_df.groupby(['merged_label']).size().to_frame('test').reset_index()

plt.figure(figsize=(8, 6))
x_axis = np.arange(len(train_df_class['merged_label']))
index = [[train_df_class,'train',-0.2],[val_df_class,'val',0],[test_df_class,'test',0.2]]
for i in index:
  df = i[0]
  bars = plt.bar(x_axis + i[2], df[i[1]] , width=0.2, label=i[1])
  for bar in bars:
      yval = bar.get_height()
      plt.text(bar.get_x(), yval + .005, yval)
plt.xticks(x_axis,train_df_class['merged_label'])
plt.legend()
plt.show()

In [ ]:
class ForestDataset(Dataset):
    """Forest dataset."""

    def __init__(self, csv_file_to_df, root_dir='../../dataset/ForestNetDataset ', transform=None, types="classifier"):
        self.csv = csv_file_to_df
        self.root_dir = root_dir
        self.transform = transform
        self.label_to_int = {'Grassland shrubland':0, 'Other':1, 'Plantation':2, 'Smallholder agriculture':3}
        self.types = types

    def __len__(self):
        return len(self.csv)

    def poly_from_utm(self, polygon):
        poly_pts = []

        poly = unary_union(polygon)
        for i in np.array(poly.exterior.coords):

            poly_pts.append(tuple(i))

        new_poly = Polygon(poly_pts)
        return new_poly

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        label = self.csv.iloc[idx, 0]
        merged_label = self.csv.iloc[idx, 1]
        lat = self.csv.iloc[idx, 2]
        long = self.csv.iloc[idx, 3]
        year = self.csv.iloc[idx, 4]
        folder = self.csv.iloc[idx, 5]

        ## Load the image and auxiliary
        image = io.imread(f'{self.root_dir}/{folder}/images/visible/composite.png')
        slope = np.load(f'{self.root_dir}/{folder}/auxiliary/slope.npy')

        ## Get the segmentation map
        with open(f'{self.root_dir}/{folder}/forest_loss_region.pkl', 'rb') as f:
            data = pickle.load(f)

        nx, ny = 332, 332
        xy_array = np.empty((0,2))
        if data.geom_type == 'Polygon':
            data = [data]
        elif data.geom_type == 'Multipolygon':
            data = list(data)

        poly_shp = []
        for poly_verts in data:
            poly_shp.append(self.poly_from_utm(poly_verts))

        mask = rasterize(shapes=poly_shp, out_shape=(332,332))
        seg = np.array(mask)

        if self.transform:
            augmented = self.transform(image=image)
            image = augmented["image"]

        image = torch.from_numpy(image).type(torch.float)
        image = image.permute(2, 0, 1)
        seg = torch.from_numpy(seg).type(torch.uint8)
        slope = torch.from_numpy(slope).type(torch.float)

        merged_label = self.label_to_int[merged_label]

        image = image[:, 86:246, 86:246]
        seg = seg[86:246, 86:246]
        slope = slope[86:246, 86:246]
        if self.types == "classifier":
            return image, slope, merged_label
        else:
            return image, merged_label


In [ ]:
transform = A.Compose(
    [
        A.HorizontalFlip(p=0.5),
        A.Rotate(270),
        A.ElasticTransform(
            p=0.4, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03
        ),
    ]
)

In [ ]:
# Initialization several arguments for training
BATCH_SIZE = 16
DEVICE = device = "cuda:0" if torch.cuda.is_available() else "cpu"
LR = 0.001
EPOCH = 20
nbClass = 4

In [ ]:
# Load Dataset and Create DataLoader For Classification
train_dataset = ForestDataset(csv_file_to_df=train_df, transform=transform)
val_dataset = ForestDataset(csv_file_to_df=val_df)
test_dataset = ForestDataset(csv_file_to_df=test_df)
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE,shuffle=True)
valloader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE,shuffle=True)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE,shuffle=False)
dataloaders = {'train': trainloader, 'val':valloader}

# Load Dataset and Create DataLoader For Representation Learning
train_dataset_re = ForestDataset( csv_file_to_df=train_df, transform=transform, types="projection")
val_dataset_re = ForestDataset( csv_file_to_df=val_df, types="projection")
test_dataset_re = ForestDataset( csv_file_to_df=test_df, types="projection")
trainloader_re = torch.utils.data.DataLoader(train_dataset_re, batch_size=BATCH_SIZE,shuffle=True)
valloader_re = torch.utils.data.DataLoader(val_dataset_re, batch_size=BATCH_SIZE,shuffle=False)
testloader_re = torch.utils.data.DataLoader(test_dataset_re, batch_size=BATCH_SIZE,shuffle=False)
dataloaders_re = {'train': trainloader_re, 'val':valloader_re}

In [ ]:
dataiter = iter(trainloader)
images, slope, labels = dataiter.next()

print('images shape on PyTroch : ', images.size())
print('slope shape on PyTroch : ', slope.size())
print('labels shape on PyTroch : ', labels.size())

In [ ]:
def show_curves(acc_dict, loss_dict):
    fig = plt.figure(figsize=(22,8))
    train_score,eval_score = acc_dict.values()
    train_score=list(map(float,train_score))
    eval_score=list(map(float,eval_score))
    ax = plt.figure().gca()
    plt.plot(train_score)
    plt.plot(eval_score)

    ax.xaxis.set_major_locator(MaxNLocator(integer=True))

    plt.legend(['train', 'val'])
    plt.xlabel('number of epochs')
    plt.ylabel('Accuracy')
    plt.title('Learning curves')
    plt.show()

    train_loss, eval_loss = loss_dict.values()
    train_loss=list(map(float,train_loss))
    eval_loss=list(map(float,eval_loss))
    ax = plt.figure().gca()
    plt.plot(train_loss)
    plt.plot(eval_loss)

    ax.xaxis.set_major_locator(MaxNLocator(integer=True))

    plt.legend(['train', 'val'])
    plt.xlabel('number of epochs')
    plt.ylabel('Loss')
    plt.title('Loss')
    plt.show()

In [ ]:
dataset_sizes = {'train':len(train_df), 'val':len(val_df)}
acc_dict = {'train':[], 'val':[]}
loss_dict= {'train':[], 'val':[]}

## Multimodal SuperCon using Custom Resnet as Backbone

In [ ]:
def train_model_SCL(model, criterion, optimizer, scheduler, num_epochs, dataloaders):
    start = time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = np.inf
    history = defaultdict(list)
    scaler = amp.GradScaler()

    for step, epoch in enumerate(range(1,num_epochs+1)):
        print('Epoch {}/{}'.format(epoch, num_epochs))
        print('-' * 10)
        for phase in ['train','val']:
            if(phase == 'train'):
                model.train()
            else:
                model.eval()

            running_loss = 0.0

            for inputs,labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                with torch.set_grad_enabled(phase == 'train'):
                    with amp.autocast(enabled=True):
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        loss = loss / 4

                    if phase == 'train':
                        scaler.scale(loss).backward()

                    if phase == 'train' and (step + 1) % 4 == 0:
                        scaler.step(optimizer)
                        scaler.update()
                        optimizer.zero_grad()
                        scheduler.step()

                running_loss += loss.item()*inputs.size(0)

            epoch_loss = running_loss/dataset_sizes[phase]
            history[phase + ' loss'].append(epoch_loss)

            print('{} Loss: {:.4f}'.format(
                phase, epoch_loss))

            # deep copy the model
            if phase=='val' and epoch_loss <= best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())

    end = time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best Loss ",best_loss)

    model.load_state_dict(best_model_wts)
    return model, history

In [ ]:
"""Training Projection Head"""

pretrained_resnet_18 = timm.create_model('resnet18', pretrained=True)
num_ftrs = pretrained_resnet_18.get_classifier().in_features
pretrained_resnet_18.fc = nn.Linear(num_ftrs, 4)
pretrained_resnet_18 = pretrained_resnet_18.to(device)

criterion = SupervisedContrastiveLoss()
optimizer = torch.optim.Adam(pretrained_resnet_18.parameters(), lr=0.001)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
model, history = train_model_SCL(pretrained_resnet_18, criterion, optimizer, scheduler, 20, dataloaders_re)


In [ ]:
plt.style.use('fivethirtyeight')
plt.rcParams["font.size"] = "15"
fig = plt.figure(figsize=(22,8))
epochs = list(range(20))

plt.plot(epochs, history['train loss'], label='train loss')
plt.plot(epochs, history['val loss'], label='valid loss')
plt.ylabel('Loss', fontsize=20)
plt.xlabel('Epoch', fontsize=20)
plt.legend()
plt.title('Loss Curve');

In [ ]:
dataset_sizes = {'train':len(train_df), 'val':len(val_df)}

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for images, slope, labels in dataloaders[phase]:
                images = images.to(device)
                slope = slope.to(device)
                labels = labels.type(torch.LongTensor).to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(images, slope).to(device)
                    _, preds = torch.max(outputs, 1)
                    preds.to(device)
                    loss = criterion(outputs.to(device), labels.to(device))

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * images.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            acc_dict[phase].append(epoch_acc)
            loss_dict[phase].append(epoch_loss)

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    print('Best val Acc: {:4f}'.format(best_acc))

    show_curves(acc_dict, loss_dict)
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
"""Training Resnet Classifier"""

resnet_classifier = resnet18(pretrained=False)

num_ftrs = resnet_classifier.fc.in_features
resnet_classifier.fc = nn.Linear(
    in_features = num_ftrs,
    out_features = nbClass
)

resnet_classifier = resnet_classifier.to(device)
criterion = FocalLoss()
optimizer = torch.optim.Adam(resnet_classifier.parameters(), lr=1e-3, weight_decay=1e-5)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
acc_dict = {'train':[], 'val':[]}
loss_dict= {'train':[], 'val':[]}

resnet_classifier = train_model(resnet_classifier, criterion, optimizer, scheduler, num_epochs=15)

In [ ]:
def test(model):
    running_accuracy = 0
    total = 0

    with torch.no_grad():
        for images, slope, labels in testloader:
            images = images.to(device)
            slope = slope.to(device)
            labels = labels.type(torch.LongTensor).to(device)
            outputs = model(images, slope).to(device)
            _, preds = torch.max(outputs, 1)
            preds.to(device)

            total += labels.size(0)
            running_accuracy += (preds == labels).sum()

        print('Accuracy of the model based on the test set is: %d %%' % (100 * running_accuracy / total))

In [ ]:
test(resnet_classifier)

## Multimodal SuperCon using Custom Efficientnet as Backbone

In [ ]:
"""Training Projection Head"""
from collections import defaultdict
from time import time

pretrained_eff_b1 = timm.create_model('efficientnet_b2', pretrained=True)
num_ftrs = pretrained_eff_b1.get_classifier().in_features
pretrained_eff_b1.fc = nn.Linear(num_ftrs, 4)
pretrained_eff_b1 = pretrained_eff_b1.to(device)

criterion = SupervisedContrastiveLoss()
optimizer = torch.optim.Adam(pretrained_eff_b1.parameters(), lr=0.001)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


model, history = train_model_SCL(pretrained_eff_b1, criterion, optimizer, scheduler, 20, dataloaders_re)

In [ ]:
epochs = list(range(20))
plt.plot(epochs, history['train loss'], label='train loss')
plt.plot(epochs, history['val loss'], label='valid loss')
plt.ylabel('Loss', fontsize=20)
plt.xlabel('Epoch', fontsize=20)
plt.legend()
plt.title('Loss Curve');

In [ ]:
"""EfficientNet B2 Classifier"""

effnet_b2_classifier = effnetb2(pretrained=False)
effnet_b2_classifier = effnet_b2_classifier.to(device)
criterion = FocalLoss()
optimizer = torch.optim.Adam(effnet_b2_classifier.parameters(), lr=1e-3, weight_decay=1e-5)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
acc_dict = {'train':[], 'val':[]}
loss_dict= {'train':[], 'val':[]}

effnet_b2_classifier = train_model(effnet_b2_classifier, criterion, optimizer, scheduler, num_epochs=15)


In [ ]:
test(effnet_b2_classifier)